# Heurística BMTSP
Este notebook implementa una heurística constructiva con mejora local para el Bounded Multiple Traveling Salesman Problem (BMTSP). Se usan los archivos CSV en `datos/` como conjunto de ciudades.

In [ ]:
import csv
import math
import random
from dataclasses import dataclass

In [ ]:
@dataclass
class City:
    idx: int
    x: float
    y: float

    def distance_to(self, other: "City") -> float:
        return math.hypot(self.x - other.x, self.y - other.y)

In [ ]:
def load_cities(path):
    cities = []
    with open(path, newline="") as f:
        reader = csv.DictReader(f)
        for row in reader:
            idx = int(row.get('id', row.get('idx')))
            cities.append(City(idx, float(row['x']), float(row['y'])))
    cities.sort(key=lambda c: c.idx)
    return cities

In [ ]:
def nearest_neighbor_route(cities, start):
    route = [start]
    remaining = cities[:]
    current = start
    while remaining:
        next_city = min(remaining, key=lambda c: current.distance_to(c))
        route.append(next_city)
        remaining.remove(next_city)
        current = next_city
    route.append(start)
    return route

In [ ]:
def route_length(route):
    return sum(route[i].distance_to(route[i+1]) for i in range(len(route)-1))

In [ ]:
def two_opt(route):
    improved = True
    best = route
    while improved:
        improved = False
        for i in range(1, len(best) - 2):
            for j in range(i + 1, len(best) - 1):
                if j - i == 1:
                    continue
                new_route = best[:i] + best[i:j][::-1] + best[j:]
                if route_length(new_route) < route_length(best):
                    best = new_route
                    improved = True
        route = best
    return best

In [ ]:
def assign_cities(cities, k, max_ciudades):
    depot = cities[0]
    others = cities[1:]
    random.shuffle(others)
    ass = [[] for _ in range(k)]
    idx = 0
    for city in others:
        ass[idx % k].append(city)
        idx += 1
    for i in range(k):
        ass[i] = ass[i][:max_ciudades]
    return ass

In [ ]:
def heuristic_bmtsp(cities, k, max_ciudades):
    depot = cities[0]
    assignments = assign_cities(cities, k, max_ciudades)
    routes = []
    total_cost = 0.0
    for assigned in assignments:
        if not assigned:
            routes.append([depot, depot])
            continue
        route = nearest_neighbor_route(assigned, depot)
        route = two_opt(route)
        routes.append(route)
        total_cost += route_length(route)
    return routes, total_cost

In [ ]:
# Ejemplo de ejecución
csv_path = 'datos/ciudades_pequeno.csv'
k = 2
max_ciudades = 5

cities = load_cities(csv_path)
routes, cost = heuristic_bmtsp(cities, k, max_ciudades)
print(f'Costo total: {cost:.2f}')
for i, route in enumerate(routes, 1):
    print(f'Vendedor {i}:', ' -> '.join(str(c.idx) for c in route))